## Load slices of binary data 

In [1]:
import numpy as np
from glob import glob
import pyqtgraph as pq
from fish.image.vol import get_stack_dims, filter_flat, unfilter_flat
%gui qt

In [2]:
# turn 4D array into 3D
def unroll(v):
    return v.reshape(v.shape[0], v.shape[1] * v.shape[2], v.shape[3])

# seed correlation
def seed_corr(seed, data):
    from numpy import where, correlate, prod, zeros
    from scipy.stats import zscore
 
    flat = data.reshape(data.shape[0], prod(data.shape[1:])).T
    valid = where(flat.std(1) != 0)[0]
    seed = zscore(seed).copy()
    normed = zscore(flat[valid, :], axis=1)
    
    corrs = zeros(flat.shape[0])
    
    for ind, val in enumerate(valid):
        corrs[val] = correlate(seed, normed[ind]) / len(seed)

    corrs = corrs.reshape(data.shape[1:])
    
    return corrs

# Need this function to prevent pyqtgraph from leaking memory
def wipe_ims(pq):
    from numpy import zeros
    for iw in pq.images:
      iw.setImage(zeros((1,1))) #<- dummy 1x1 image
      iw.parent().close()

In [3]:
base_dir = 'F:/davis/20170501/5dpf_cy302_f1_omr_2_20170501_204050/'
dims = tuple(get_stack_dims(base_dir)[::-1])
fnames = glob(base_dir + '*.stack')
fnames.sort()
print(np.r_[len(fnames), dims])

mem = [np.memmap(f, dtype='uint16', shape=dims) for f in fnames]

[6000   11  912  864]


In [4]:
%%time
wipe_ims(pq)
plr = slice(0,6000)
dat = np.array([m[5,:,:] for m in mem[plr]])

Wall time: 1min


In [5]:
if dat.ndim > 3:
    im = pq.image(unroll(dat))
else:
    im = pq.image(dat)

In [ ]:
rois = {}

In [ ]:
rois['motor'] = im.roi.getArrayRegion(dat, im.imageItem, axes=(1,2)).mean((1,2))

In [ ]:
from scipy.stats import zscore
pl = pq.plot(cam, 5 + zscore(rois['motor']), color='m')
pl.plot(zscore(epdat[0]), color='y')

In [ ]:
# load ephys
from fish.ephys.ephys import load, windowed_variance
ep_fname = 'W:/davis/data/ephys/20170502/6dpf_cy302xcy171_f2_omr_2.10chFlt'
epdat = load(ep_fname)

In [ ]:
from fish.ephys.ephys import estimate_onset

In [ ]:
cam = estimate_onset(epdat[2], 3.7, 100)[:len(fnames)]

In [ ]:
smth = [windowed_variance(v)[0] for v in epdat[0:2]]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib
plt.plot(smth[1])
plt.show()

In [ ]:
pq.plot(epdat[1][cam[0]:cam[-1]][::10])